# Assignment 3

## Task 1

I wish to solve the 2D Poisson equation 

$$\nabla u = f(x,y) \quad \mathrm{for} \quad 0<x,y<1 \\ u = 0 \quad \mathrm{on \: the \: boundary.} $$

To do this, I start by taking use of the 5-point scheme

$$ U_{i+1, j} + U_{i-1, j} + U_{i, j+1} + U_{i, j-1} - 4U_{i,j}= h^2 f_{i,j} $$

which can be written as a system of equations $\mathbf{A} \mathbf{u} = h^2 \mathbf{f}$. The $\mathbf{A}$ matrix can be implemented as below.

In [8]:
import numpy as np
import math

def five_point(N):
    u = np.eye(N**2)
    Au = 4 * u

    Au[:,:-N] = Au[:,:-N] - u[:,N:]
    Au[:-N,:] = Au[:-N,:] - u[N:,:]

    diag = np.ones(N)
    diag[-1] = 0
    np.fill_diagonal(u, diag)

    Au[1:,:] = Au[1:,:] - u[:-1,:]

    diag = np.ones(N)
    diag[0] = 0
    np.fill_diagonal(u, diag)

    Au[:-1,:] = Au[:-1,:] - u[1:,:]

    return -Au


print(five_point(4))

[[-4.  1. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [ 1. -4.  1. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0.  1. -4.  1. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0. -0.]
 [-0. -0.  1. -4. -0. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0. -0.]
 [ 1. -0. -0. -0. -4.  1. -0. -0.  1. -0. -0. -0. -0. -0. -0. -0.]
 [-0.  1. -0. -0.  1. -4.  1. -0. -0.  1. -0. -0. -0. -0. -0. -0.]
 [-0. -0.  1. -0. -0.  1. -4.  1. -0. -0.  1. -0. -0. -0. -0. -0.]
 [-0. -0. -0.  1. -0. -0.  1. -4. -0. -0. -0.  1. -0. -0. -0. -0.]
 [-0. -0. -0. -0.  1. -0. -0. -0. -4.  1. -0. -0.  1. -0. -0. -0.]
 [-0. -0. -0. -0. -0.  1. -0. -0.  1. -4.  1. -0. -0.  1. -0. -0.]
 [-0. -0. -0. -0. -0. -0.  1. -0. -0.  1. -4.  1. -0. -0.  1. -0.]
 [-0. -0. -0. -0. -0. -0. -0.  1. -0. -0.  1. -4. -0. -0. -0.  1.]
 [-0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0. -0. -4.  1. -0. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0.  1. -4.  1. -0.]
 [-0. -0. -0. -0. -0. -0. -0. -0. -0. -0.  1. -0. -0.  1. -4. 

As seen above, the five-point stencil is a tridiagonal block matrix. The right-hand side of this scheme, $\mathbf{f}$, is 

In [4]:
def calc_rhs(N):
    N2 = N**2

    p = np.zeros((N2, 2))
    for i in range(1, N2):
        p[i, 0] = (i % N)/(N-1)
        p[i, 1] = math.floor(i/N)/(N-1)

    f = lambda x,y: math.sin(x*y)

    b = np.zeros(N2)
    for i in range(N2):
        b[i] = f(p[i,0], p[i,1])

    # Boundary conditions
    b[0:N] = 0                  # y=0
    b[N2-N:N2] = 0              # y=1
    b[0:N2:N] = 0               # x=0
    b[N-1:N2:N] = 0             # x=1

    b = np.reshape(b, (N, N))
    return b

N = 5
print(rhs(N))

[[0.         0.         0.         0.         0.        ]
 [0.         0.06245932 0.12467473 0.1864033  0.        ]
 [0.         0.12467473 0.24740396 0.36627253 0.        ]
 [0.         0.1864033  0.36627253 0.53330267 0.        ]
 [0.         0.         0.         0.         0.        ]]


as the given boundary conditions are 0. I wish to solve this system of equations with a V-cycle multigrid method, so I start by implementing a weighted Jacobi method. This iterative method takes the form

$$u_{j+1} = [(1-\omega)I + \omega D^{-1}(D-A)]u_j + \omega D^{-1} f = J_{\omega} u_j + f_{\omega}.$$

Since for the five-point method the diagonal matrix will be $D = \frac{1}{4}I$, I can rewrite $J_{\omega}$ as

$$ J_\omega = I - \frac{1}{4} \omega A,$$

and the implementation can be done as such

In [6]:
def jacobi(u0, rhs, w, nu):
    """
    u0: Initial guess
    rhs: Right-hand side, f
    w: Weights
    nu: Number of presmoothings
    """
    # Initializing variables
    N2 = u0.shape[0]
    N = int(np.sqrt(N2))
    nu = nu + 1

    I = np.eye(N2)
    A = five_point(N)
    Jw = I - 1/4 * w * A

    # Calculating the weighted Jacobi iteration
    u = Jw.dot(u0) + w * 1/4 * I * rhs

    return u, nu

$$ \nabla^2 = u $$